In [1]:
import math
import statistics
import random
import utils
from itertools import combinations

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%load_ext memray

In [3]:
_pretty_suits = ["♥", "♦", "♠", "♣"]
_boring_suits = ["H", "D", "S", "C"]

_values = list(range(2, 10))
_values.extend(["T", "J", "Q", "K", "A"])

pretty_deck = []
deck = []
for i in range(len(_pretty_suits)):
    for value in _values:
        pretty_deck.append(f"{value}{_pretty_suits[i]}")
        deck.append(f"{value}{_boring_suits[i]}")

In [4]:
print(len(deck))
print(deck[0:14])

52
['2H', '3H', '4H', '5H', '6H', '7H', '8H', '9H', 'TH', 'JH', 'QH', 'KH', 'AH', '2D']


In [5]:
starting_hand_size = 6
hand_size = 4

In [6]:
# 52c4 * 48
# where 48 is the remaining cards after you get your hand
unique_hands = math.comb(len(deck), hand_size) * (len(deck) - hand_size)
print(f"{unique_hands=:,}")

# 6 cards dealt to you, you have to discard 2
unique_hands_dealt = math.comb(len(deck), starting_hand_size)
print(f"{unique_hands_dealt=:,}")

# from the 6 dealt, you could discard any 2 of them
unique_sub_hands = math.comb(starting_hand_size, hand_size)
print(f"{unique_sub_hands=:,}")

# this means you can have any combination of 52c6 * 6c2 number of hands
total_possible_hands = unique_hands_dealt * unique_sub_hands
print(f"{total_possible_hands=:,}")

# crib will be made up of (52-6)c2 * 2 of your 6 cards (or 6c2)
possible_crib_hands = math.comb(len(deck) - starting_hand_size, 2) * unique_sub_hands
print(f"{possible_crib_hands=:,}")

unique_hands=12,994,800
unique_hands_dealt=20,358,520
unique_sub_hands=15
total_possible_hands=305,377,800
possible_crib_hands=15,525


In [7]:
# set of all possible 6 card starting hands
starting_hand_list = list(combinations(deck, r=starting_hand_size))
# proof of no duplicates for peace of mind
starting_hand_set = frozenset(starting_hand_list)

In [8]:
print(f"{len(starting_hand_list)=:,}")
print(f"{starting_hand_list[:10]}")

len(starting_hand_list)=20,358,520
[('2H', '3H', '4H', '5H', '6H', '7H'), ('2H', '3H', '4H', '5H', '6H', '8H'), ('2H', '3H', '4H', '5H', '6H', '9H'), ('2H', '3H', '4H', '5H', '6H', 'TH'), ('2H', '3H', '4H', '5H', '6H', 'JH'), ('2H', '3H', '4H', '5H', '6H', 'QH'), ('2H', '3H', '4H', '5H', '6H', 'KH'), ('2H', '3H', '4H', '5H', '6H', 'AH'), ('2H', '3H', '4H', '5H', '6H', '2D'), ('2H', '3H', '4H', '5H', '6H', '3D')]


In [16]:
starting_hands_with_possible_sub_hands = {
    element: list(combinations(element, r=hand_size)) for element in starting_hand_list
}

In [18]:
starting_hands_with_possible_sub_hands[("2H", "3H", "4H", "5H", "6H", "7H")])

[('2H', '3H', '4H', '5H'),
 ('2H', '3H', '4H', '6H'),
 ('2H', '3H', '4H', '7H'),
 ('2H', '3H', '5H', '6H'),
 ('2H', '3H', '5H', '7H'),
 ('2H', '3H', '6H', '7H'),
 ('2H', '4H', '5H', '6H'),
 ('2H', '4H', '5H', '7H'),
 ('2H', '4H', '6H', '7H'),
 ('2H', '5H', '6H', '7H'),
 ('3H', '4H', '5H', '6H'),
 ('3H', '4H', '5H', '7H'),
 ('3H', '4H', '6H', '7H'),
 ('3H', '5H', '6H', '7H'),
 ('4H', '5H', '6H', '7H')]

In [19]:
df_starting_hands = pd.DataFrame(
    starting_hands_with_possible_sub_hands.items(),
    columns=["starting_hand", "sub_hands"],
)

In [21]:
df_starting_hands.head()

,starting_hand,sub_hands
0,"(2H, 3H, 4H, 5H, 6H, 7H)","[(2H, 3H, 4H, 5H), (2H, 3H, 4H, 6H), (2H, 3H, ..."
1,"(2H, 3H, 4H, 5H, 6H, 8H)","[(2H, 3H, 4H, 5H), (2H, 3H, 4H, 6H), (2H, 3H, ..."
2,"(2H, 3H, 4H, 5H, 6H, 9H)","[(2H, 3H, 4H, 5H), (2H, 3H, 4H, 6H), (2H, 3H, ..."
3,"(2H, 3H, 4H, 5H, 6H, TH)","[(2H, 3H, 4H, 5H), (2H, 3H, 4H, 6H), (2H, 3H, ..."
4,"(2H, 3H, 4H, 5H, 6H, JH)","[(2H, 3H, 4H, 5H), (2H, 3H, 4H, 6H), (2H, 3H, ..."


In [23]:
df_starting_hands["starting_hand_str"] = df_starting_hands.apply(
    lambda row: ",".join(str(x) for x in row["starting_hand"]), axis=1
)

df_starting_hands.head()

,starting_hand,sub_hands,starting_hand_str
0,"(2H, 3H, 4H, 5H, 6H, 7H)","[(2H, 3H, 4H, 5H), (2H, 3H, 4H, 6H), (2H, 3H, ...","2H,3H,4H,5H,6H,7H"
1,"(2H, 3H, 4H, 5H, 6H, 8H)","[(2H, 3H, 4H, 5H), (2H, 3H, 4H, 6H), (2H, 3H, ...","2H,3H,4H,5H,6H,8H"
2,"(2H, 3H, 4H, 5H, 6H, 9H)","[(2H, 3H, 4H, 5H), (2H, 3H, 4H, 6H), (2H, 3H, ...","2H,3H,4H,5H,6H,9H"
3,"(2H, 3H, 4H, 5H, 6H, TH)","[(2H, 3H, 4H, 5H), (2H, 3H, 4H, 6H), (2H, 3H, ...","2H,3H,4H,5H,6H,TH"
4,"(2H, 3H, 4H, 5H, 6H, JH)","[(2H, 3H, 4H, 5H), (2H, 3H, 4H, 6H), (2H, 3H, ...","2H,3H,4H,5H,6H,JH"


In [24]:
# df2[['team1','team2']] = pd.DataFrame(df2.teams.tolist(), index= df2.index)
sub_hands_col_names = [f"sub_hands_str_{i}" for i in range(unique_sub_hands)]

df_starting_hands[sub_hands_col_names] = pd.DataFrame(
    df_starting_hands.sub_hands.tolist(), index=df_starting_hands.index
)

In [25]:
df_starting_hands.head()

,starting_hand,sub_hands,starting_hand_str,sub_hands_str_0,sub_hands_str_1,sub_hands_str_2,sub_hands_str_3,sub_hands_str_4,sub_hands_str_5,sub_hands_str_6,sub_hands_str_7,sub_hands_str_8,sub_hands_str_9,sub_hands_str_10,sub_hands_str_11,sub_hands_str_12,sub_hands_str_13,sub_hands_str_14
0,"(2H, 3H, 4H, 5H, 6H, 7H)","[(2H, 3H, 4H, 5H), (2H, 3H, 4H, 6H), (2H, 3H, ...","2H,3H,4H,5H,6H,7H","(2H, 3H, 4H, 5H)","(2H, 3H, 4H, 6H)","(2H, 3H, 4H, 7H)","(2H, 3H, 5H, 6H)","(2H, 3H, 5H, 7H)","(2H, 3H, 6H, 7H)","(2H, 4H, 5H, 6H)","(2H, 4H, 5H, 7H)","(2H, 4H, 6H, 7H)","(2H, 5H, 6H, 7H)","(3H, 4H, 5H, 6H)","(3H, 4H, 5H, 7H)","(3H, 4H, 6H, 7H)","(3H, 5H, 6H, 7H)","(4H, 5H, 6H, 7H)"
1,"(2H, 3H, 4H, 5H, 6H, 8H)","[(2H, 3H, 4H, 5H), (2H, 3H, 4H, 6H), (2H, 3H, ...","2H,3H,4H,5H,6H,8H","(2H, 3H, 4H, 5H)","(2H, 3H, 4H, 6H)","(2H, 3H, 4H, 8H)","(2H, 3H, 5H, 6H)","(2H, 3H, 5H, 8H)","(2H, 3H, 6H, 8H)","(2H, 4H, 5H, 6H)","(2H, 4H, 5H, 8H)","(2H, 4H, 6H, 8H)","(2H, 5H, 6H, 8H)","(3H, 4H, 5H, 6H)","(3H, 4H, 5H, 8H)","(3H, 4H, 6H, 8H)","(3H, 5H, 6H, 8H)","(4H, 5H, 6H, 8H)"
2,"(2H, 3H, 4H, 5H, 6H, 9H)","[(2H, 3H, 4H, 5H), (2H, 3H, 4H, 6H), (2H, 3H, ...","2H,3H,4H,5H,6H,9H","(2H, 3H, 4H, 5H)","(2H, 3H, 4H, 6H)","(2H, 3H, 4H, 9H)","(2H, 3H, 5H, 6H)","(2H, 3H, 5H, 9H)","(2H, 3H, 6H, 9H)","(2H, 4H, 5H, 6H)","(2H, 4H, 5H, 9H)","(2H, 4H, 6H, 9H)","(2H, 5H, 6H, 9H)","(3H, 4H, 5H, 6H)","(3H, 4H, 5H, 9H)","(3H, 4H, 6H, 9H)","(3H, 5H, 6H, 9H)","(4H, 5H, 6H, 9H)"
3,"(2H, 3H, 4H, 5H, 6H, TH)","[(2H, 3H, 4H, 5H), (2H, 3H, 4H, 6H), (2H, 3H, ...","2H,3H,4H,5H,6H,TH","(2H, 3H, 4H, 5H)","(2H, 3H, 4H, 6H)","(2H, 3H, 4H, TH)","(2H, 3H, 5H, 6H)","(2H, 3H, 5H, TH)","(2H, 3H, 6H, TH)","(2H, 4H, 5H, 6H)","(2H, 4H, 5H, TH)","(2H, 4H, 6H, TH)","(2H, 5H, 6H, TH)","(3H, 4H, 5H, 6H)","(3H, 4H, 5H, TH)","(3H, 4H, 6H, TH)","(3H, 5H, 6H, TH)","(4H, 5H, 6H, TH)"
4,"(2H, 3H, 4H, 5H, 6H, JH)","[(2H, 3H, 4H, 5H), (2H, 3H, 4H, 6H), (2H, 3H, ...","2H,3H,4H,5H,6H,JH","(2H, 3H, 4H, 5H)","(2H, 3H, 4H, 6H)","(2H, 3H, 4H, JH)","(2H, 3H, 5H, 6H)","(2H, 3H, 5H, JH)","(2H, 3H, 6H, JH)","(2H, 4H, 5H, 6H)","(2H, 4H, 5H, JH)","(2H, 4H, 6H, JH)","(2H, 5H, 6H, JH)","(3H, 4H, 5H, 6H)","(3H, 4H, 5H, JH)","(3H, 4H, 6H, JH)","(3H, 5H, 6H, JH)","(4H, 5H, 6H, JH)"


In [21]:
points_df = pd.read_csv("crib_hands.csv")

# making columns easier to work with
points_df["hand"] = (
    points_df["hand_card_1"]
    + ","
    + points_df["hand_card_2"]
    + ","
    + points_df["hand_card_3"]
    + ","
    + points_df["hand_card_4"]
    + ","
    + points_df["cut_card"]
)
points_df = points_df.drop(
    columns=["hand_card_1", "hand_card_2", "hand_card_3", "hand_card_4", "cut_card"]
)

points_df.set_index("hand", inplace=True)

points_dict = points_df.T.to_dict(
    orient="list",
)

In [22]:
points_dict

{'2H,3H,4H,5H,6H': [14],
 '2H,3H,4H,5H,7H': [11],
 '2H,3H,4H,5H,8H': [13],
 '2H,3H,4H,5H,9H': [11],
 '2H,3H,4H,5H,TH': [13],
 '2H,3H,4H,5H,JH': [13],
 '2H,3H,4H,5H,QH': [13],
 '2H,3H,4H,5H,KH': [13],
 '2H,3H,4H,5H,AH': [12],
 '2H,3H,4H,5H,2D': [14],
 '2H,3H,4H,5H,3D': [16],
 '2H,3H,4H,5H,4D': [16],
 '2H,3H,4H,5H,5D': [16],
 '2H,3H,4H,5H,6D': [13],
 '2H,3H,4H,5H,7D': [10],
 '2H,3H,4H,5H,8D': [12],
 '2H,3H,4H,5H,9D': [10],
 '2H,3H,4H,5H,TD': [12],
 '2H,3H,4H,5H,JD': [12],
 '2H,3H,4H,5H,QD': [12],
 '2H,3H,4H,5H,KD': [12],
 '2H,3H,4H,5H,AD': [7],
 '2H,3H,4H,5H,2S': [14],
 '2H,3H,4H,5H,3S': [16],
 '2H,3H,4H,5H,4S': [16],
 '2H,3H,4H,5H,5S': [16],
 '2H,3H,4H,5H,6S': [13],
 '2H,3H,4H,5H,7S': [10],
 '2H,3H,4H,5H,8S': [12],
 '2H,3H,4H,5H,9S': [10],
 '2H,3H,4H,5H,TS': [12],
 '2H,3H,4H,5H,JS': [12],
 '2H,3H,4H,5H,QS': [12],
 '2H,3H,4H,5H,KS': [12],
 '2H,3H,4H,5H,AS': [7],
 '2H,3H,4H,5H,2C': [14],
 '2H,3H,4H,5H,3C': [16],
 '2H,3H,4H,5H,4C': [16],
 '2H,3H,4H,5H,5C': [16],
 '2H,3H,4H,5H,6C': [13],
 '

In [23]:
points_df.head()

,points
hand,
"2H,3H,4H,5H,6H",14
"2H,3H,4H,5H,7H",11
"2H,3H,4H,5H,8H",13
"2H,3H,4H,5H,9H",11
"2H,3H,4H,5H,TH",13


In [40]:
# points_df.loc[(points_df["hand"] == "2H,3H,4H,6H")]
int(
    points_df.loc[
        (points_df["hand"] == "2H,3H,4H,5H") & (points_df["cut_card"] == "4C")
    ].points.values[0]
)

16

In [46]:
class PersonalHand:
    def __init__(self, cards: tuple[str]):
        self.cards = cards
        self.cards_str = ",".join(cards)
        self.cards_set = set(cards)


class PlayedHand:
    def __init__(self, hand_cards: PersonalHand, cut_card: str):
        self.hand_cards = hand_cards
        self.cut_card = cut_card
        self.points: int = self._get_points(hand_cards, cut_card)

    def _get_points(self, hand_cards: PersonalHand, cut_card: str):
        # TODO
        # val = points_df.loc[
        #     (points_df["hand"] == hand_cards.cards_str)
        #     & (points_df["cut_card"] == cut_card)
        # ].points.values[0]

        hand = hand_cards.cards_str + "," + cut_card

        try:
            val = points_dict[hand]
        except KeyError:
            print(hand)
            raise

        return int(val[0])


class PossibleHand:
    def __init__(self, four_card_hand: PersonalHand, possible_crib_cards: tuple[str]):
        self.hand = four_card_hand
        self.possible_crib_cards = possible_crib_cards

        self.possible_played_hands: list[PlayedHand] = []
        self.possible_points: list[int] = []
        self.points_max: int = 0
        self.points_min: int = 29
        for cut_card in possible_crib_cards:
            played_hand = PlayedHand(four_card_hand, cut_card)
            self.possible_played_hands.append(played_hand)
            self.possible_points.append(played_hand.points)

            if played_hand.points > self.points_max:
                self.points_max = played_hand.points
            if played_hand.points < self.points_min:
                self.points_min = played_hand.points

        # self.possible_points = [hand.points for hand in self.possible_played_hands]

        self.points_ev = statistics.mean(self.possible_points)
        self.points_stdev = statistics.stdev(self.possible_points)
        self.points_median = statistics.median(self.possible_points)
        # self.points_max = max(self.possible_points)
        # self.points_min = min(self.possible_points)


class DealtHand:
    def __init__(self, cards: tuple[str]):
        self.cards = cards

        _copied_deck = deck.copy()

        for card in cards:
            _copied_deck.remove(card)

        self.possible_crib_cards = _copied_deck
        _possible_hands_tuples = list(combinations(cards, r=hand_size))
        self.possible_personal_hands = [
            PersonalHand(hand) for hand in _possible_hands_tuples
        ]

        self.possible_hands = [
            PossibleHand(hand, self.possible_crib_cards)
            for hand in self.possible_personal_hands
        ]

        self.ev_hand = max(self.possible_hands, key=lambda hand: hand.points_ev)
        self.max_hand = max(self.possible_hands, key=lambda hand: hand.points_max)
        self.median_hand = max(self.possible_hands, key=lambda hand: hand.points_median)

In [45]:
# example_hand = ("2H", "3H", "4H", "5H", "6H", "7H")
# example_hand_obj = DealtHand(example_hand)

In [52]:
# example_hand_2 = ("4H", "TH", "JH", "2D", "TD", "8S")
# example_hand_2_obj = DealtHand(example_hand_2)

In [36]:
# %%memray_flamegraph

from cProfile import Profile
from pstats import Stats, SortKey

with Profile() as pr:
    example_hand_3 = ("4H", "TH", "JH", "2D", "9D", "8S")
    example_hand_3_obj = DealtHand(example_hand_3)

    (Stats(pr).strip_dirs().sort_stats(SortKey.CUMULATIVE).print_stats())

         8790 function calls (8785 primitive calls) in 0.088 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      2/1    0.000    0.000    0.049    0.049 {method 'run' of '_contextvars.Context' objects}
        1    0.000    0.000    0.049    0.049 asyncio.py:200(_handle_events)
        1    0.000    0.000    0.049    0.049 zmqstream.py:584(_handle_events)
        1    0.000    0.000    0.048    0.048 zmqstream.py:625(_handle_recv)
        1    0.000    0.000    0.048    0.048 zmqstream.py:557(_run_callback)
        1    0.000    0.000    0.048    0.048 iostream.py:157(_handle_event)
      2/1    0.022    0.011    0.048    0.048 1572010496.py:61(__init__)
    16/15    0.002    0.000    0.038    0.003 1572010496.py:33(__init__)
  691/690    0.002    0.000    0.019    0.000 1572010496.py:9(__init__)
      3/2    0.018    0.006    0.018    0.009 events.py:86(_run)
       15    0.001    0.000    0.016    0.001 statistics.py:

In [37]:
print(f"{example_hand_3_obj.max_hand.hand.cards_str=}")
print(f"{example_hand_3_obj.ev_hand.hand.cards_str=}")
print(f"{example_hand_3_obj.median_hand.hand.cards_str=}")

example_hand_3_obj.best_possible_hand_by_max.hand.cards_str='TH,JH,9D,8S'
example_hand_3_obj.best_possible_hand_by_ev.hand.cards_str='TH,JH,9D,8S'
example_hand_3_obj.best_possible_hand_by_median.hand.cards_str='TH,JH,9D,8S'


In [40]:
df = pd.DataFrame(
    columns=[
        "dealt_hand",
        "max_hand",
        "max_hand_ev_points",
        "max_hand_max_points",
        "max_hand_median_points",
        "max_hand_stdev_points",
        "median_hand",
        "median_hand_ev_points",
        "median_hand_max_points",
        "median_hand_median_points",
        "median_hand_stdev_points",
        "ev_hand",
        "ev_hand_ev_points",
        "ev_hand_max_points",
        "ev_hand_median_points",
        "ev_hand_stdev_points",
    ]
)

In [52]:
from tqdm import tqdm

for starting_hand in tqdm(starting_hand_list):
    staring_dealt_hand: DealtHand = DealtHand(starting_hand)

    row = [
        ",".join(staring_dealt_hand.cards),
    ]

    best_hands = [
        staring_dealt_hand.max_hand,
        staring_dealt_hand.median_hand,
        staring_dealt_hand.ev_hand,
    ]

    for best_hand in best_hands:
        row.append(best_hand.hand.cards_str)
        row.append(best_hand.points_ev)
        row.append(best_hand.points_max)
        row.append(best_hand.points_median)
        row.append(best_hand.points_stdev)

    # df = pd.concat([pd.DataFrame([[1,2]], columns=df.columns), df], ignore_index=True)
    df = pd.concat([pd.DataFrame([row], columns=df.columns), df], ignore_index=True)

  0%|          | 0/20358520 [00:00<?, ?it/s]/var/folders/jb/2g0yf8sx4cs6_kt_mtlb1z3m0000gn/T/ipykernel_48213/1575519737.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([pd.DataFrame([row], columns=df.columns), df], ignore_index=True)
  1%|          | 112299/20358520 [07:54<40:16:00, 139.67it/s]Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x106da33e0>>
Traceback (most recent call last):
  File "/Users/ianpaul/code/cribbage-hand-statistics/venv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
  1%|          | 113022/20358520 [08:00<48:01:38, 1